In [6]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import KFold
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from keras_tuner import HyperModel, Hyperband
np.random.seed(42)
tf.random.set_seed(42)

def cal_acc(df_test, labels):
    # add labels to test data
    df_test['pred'] = labels

    # calculate accuracy
    correct = 0
    for index, row in df_test.iterrows():
        if row['pred'] == row['label']:
            correct += 1
    accuracy = correct / len(df_test)
    print('Accuracy average: {}'.format(accuracy))

    # calculate accuracy per class
    correct_funny = 0
    correct_somewhat_funny = 0
    correct_not_funny = 0
    total_funny = 0
    total_somewhat_funny = 0
    total_not_funny = 0

    for index, row in df_test.iterrows():
        if row['label'] == 2:
            total_funny += 1
            if row['pred'] == 2:
                correct_funny += 1
        elif row['label'] == 1:
            total_somewhat_funny += 1
            if row['pred'] == 1:
                correct_somewhat_funny += 1
        elif row['label'] == 0:
            total_not_funny += 1
            if row['pred'] == 0:
                correct_not_funny += 1

    accuracy_funny = correct_funny / total_funny
    accuracy_somewhat_funny = correct_somewhat_funny / total_somewhat_funny
    accuracy_not_funny = correct_not_funny / total_not_funny

    print('Accuracy funny: {}'.format(accuracy_funny))
    print('Accuracy somewhat funny: {}'.format(accuracy_somewhat_funny))
    print('Accuracy not funny: {}'.format(accuracy_not_funny))

In [7]:
# Load train data
df_train = pd.read_csv('data/train_set_850-870_top30_features.csv')
data_train = df_train.drop(['mean', 'label'], axis=1)
print(data_train.shape)

# Load test data
df_test = pd.read_csv('data/test_set_850-870_top30_features.csv')
data_test = df_test.drop(['mean', 'label'], axis=1)

(2985, 30)


In [8]:
# check 'funny' exits in label
if 'funny' in df_train['label'].unique():
    # recode labels
    df_train['label'] = df_train['label'].replace('funny', 2)
    df_train['label'] = df_train['label'].replace('somewhat_funny', 1)
    df_train['label'] = df_train['label'].replace('not_funny', 0)
    df_test['label'] = df_test['label'].replace('funny', 2)
    df_test['label'] = df_test['label'].replace('somewhat_funny', 1)
    df_test['label'] = df_test['label'].replace('not_funny', 0)
else:
    pass

# Converting Labels to Categorical in train set
y_train_categorical = df_train['label']
print(y_train_categorical)

0       2
1       2
2       2
3       2
4       2
       ..
2980    0
2981    0
2982    0
2983    0
2984    0
Name: label, Length: 2985, dtype: int64


### Hyperparameter tuning

In [11]:
class MyHyperModel(HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        model = Sequential()
        model.add(Dense(units=hp.Int('unit', min_value=64, max_value=256, step=32),
                        activation=hp.Choice('activation', values=['relu', 'tanh']),
                        input_shape=self.input_shape,
                        kernel_regularizer=tf.keras.regularizers.l1_l2(l1=hp.Choice('l1', values=[0.01, 0.001, 0.0001, 0.0]),
                                                                        l2=hp.Choice('l2', values=[0.01, 0.001, 0.0001, 0.0]))))
        model.add(BatchNormalization())
        model.add(Dropout(hp.Float('dropout', min_value=0.0, max_value=0.5, step=0.1)))

        model.add(Dense(self.num_classes, activation='softmax'))

        model.compile(optimizer=tf.keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
                        loss='sparse_categorical_crossentropy',
                        metrics=['accuracy'])
        
        return model

In [12]:
# Prepare your data
X = data_train
y = y_train_categorical

# K-Fold Cross-Validation
num_folds = 5
kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)

best_hyperparams_per_fold = []

for fold, (train_indices, val_indices) in enumerate(kf.split(X)):
    print(f"Running tuning for fold {fold + 1}")

    # Split data into train and validation for the current fold
    X_train, X_val = X.loc[train_indices,], X.loc[val_indices,]
    y_train, y_val = y.loc[train_indices,], y.loc[val_indices,]

    # Define the hypermodel
    hypermodel = MyHyperModel(input_shape=X_train.shape[1:], num_classes=3)

    # Initialize the Hyperband tuner
    tuner = Hyperband(
        hypermodel,
        objective='val_accuracy',
        max_epochs=20,
        directory=f'result/my_dir_{fold}',
        project_name='hyperparameter_tuning',
        overwrite=True
    )

    # Start the tuning process
    tuner.search(X_train, y_train, validation_data=(X_val, y_val), class_weight={0: 1, 1: 1, 2: 1})

    # Store the top 3 best hyperparameters of this fold
    top_3_hyperparams_per_fold = tuner.get_best_hyperparameters(num_trials=3)
    best_hyperparams_per_fold.append(top_3_hyperparams_per_fold)

Trial 30 Complete [00h 00m 03s]
val_accuracy: 0.35343384742736816

Best val_accuracy So Far: 0.4036850929260254
Total elapsed time: 00h 00m 40s


In [13]:
# select best hyperparameters from best_hyperparams_per_fold
best_hyperparams = []
for top_hyperparams in best_hyperparams_per_fold:
    best_hyperparams.append(top_hyperparams[0])

# show best hyperparameters
print("Best hyperparameters:")
for top_hyperparam in best_hyperparams:
    print(top_hyperparam.values)

Best hyperparameters:
{'unit': 96, 'activation': 'relu', 'l1': 0.001, 'l2': 0.0, 'dropout': 0.30000000000000004, 'learning_rate': 0.001, 'tuner/epochs': 7, 'tuner/initial_epoch': 0, 'tuner/bracket': 1, 'tuner/round': 0}
{'unit': 160, 'activation': 'relu', 'l1': 0.0001, 'l2': 0.0, 'dropout': 0.30000000000000004, 'learning_rate': 0.001, 'tuner/epochs': 20, 'tuner/initial_epoch': 0, 'tuner/bracket': 0, 'tuner/round': 0}
{'unit': 192, 'activation': 'tanh', 'l1': 0.001, 'l2': 0.0001, 'dropout': 0.4, 'learning_rate': 0.001, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 'tuner/trial_id': '0012'}
{'unit': 192, 'activation': 'relu', 'l1': 0.0001, 'l2': 0.0, 'dropout': 0.4, 'learning_rate': 0.001, 'tuner/epochs': 3, 'tuner/initial_epoch': 0, 'tuner/bracket': 2, 'tuner/round': 0}
{'unit': 96, 'activation': 'relu', 'l1': 0.0, 'l2': 0.01, 'dropout': 0.2, 'learning_rate': 0.001, 'tuner/epochs': 20, 'tuner/initial_epoch': 7, 'tuner/bracket': 2, 'tuner/round': 2, 

### Test

In [14]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Build the model with the optimal hyperparameters
model = hypermodel.build(best_hyperparams[0])

# fit model
model.fit(data_train, y_train_categorical, epochs=50, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/50
75/75 [==============================] - 1s 2ms/step - loss: 1.7406 - accuracy: 0.3476 - val_loss: 2.5332 - val_accuracy: 0.0586
Epoch 2/50
75/75 [==============================] - 0s 1ms/step - loss: 1.5233 - accuracy: 0.3924 - val_loss: 2.4737 - val_accuracy: 0.0168
Epoch 3/50
75/75 [==============================] - 0s 1ms/step - loss: 1.4287 - accuracy: 0.4129 - val_loss: 1.2861 - val_accuracy: 0.3518
Epoch 4/50
75/75 [==============================] - 0s 1ms/step - loss: 1.3846 - accuracy: 0.4058 - val_loss: 1.9238 - val_accuracy: 0.0385
Epoch 5/50
75/75 [==============================] - 0s 1ms/step - loss: 1.3369 - accuracy: 0.4146 - val_loss: 2.0306 - val_accuracy: 0.0184
Epoch 6/50
75/75 [==============================] - 0s 1ms/step - loss: 1.3031 - accuracy: 0.4322 - val_loss: 1.8484 - val_accuracy: 0.0318
Epoch 7/50
75/75 [==============================] - 0s 1ms/step - loss: 1.3058 - accuracy: 0.4146 - val_loss: 2.2577 - val_accuracy: 0.0101
Epoch 8/50
75/75 [==

In [15]:
# test model
predictions = model.predict(data_test)

# transform predictions to labels
labels = []
for prediction in predictions:
    labels.append(prediction.argmax())

# calculate accuracy
cal_acc(df_test, labels)

24/24 [==============================] - 0s 649us/step
Accuracy average: 0.3507362784471218
Accuracy funny: 0.012048192771084338
Accuracy somewhat funny: 0.7028112449799196
Accuracy not funny: 0.3373493975903614


In [17]:
labels

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,


In [333]:
# write df_test columns 'pred' and 'label' to csv
# df_test[['pred', 'label']].to_csv('result/nn_predictions.csv', index=False)